<a href="https://colab.research.google.com/github/AlbertoRoldanSastre/AlbertoRoldanSastre/blob/main/YOLOV8_c%C3%B3digo_31_3_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Código para el entrenamiento del modelo YOLOv8 ( n, s, m, l, x)

# 0. Memoria ram y capacidad de memoria

Vamos a comprobar que memoria ram y capacidad estamos usando en Colab

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


# 1. Chequear si tengo GPU y cual

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# 2. Importar librerías

Importamos librerías importantes para el proceso

In [1]:
#0. En Colab, primero instala los paquetes y monta tu Drive:  Instalar las librerías necesarias
!pip install ultralytics wandb  # Esto instalará ultralytics y wandb (torch y torchvision generalmente ya vienen instalados en Colab)

import ultralytics
ultralytics.checks()

Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 41.2/235.7 GB disk)


# 3. Conectar con Google Drive

In [4]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')           #cambiar la ruta de drive de donde lo extraes

Mounted at /content/drive


#4. Importar archivos

In [8]:
import os

# Rutas de las dos carpetas
folder1 = "/content/drive/MyDrive/AWS.v39i.seals+holes/ready/va/images"
folder2 = "/content/drive/MyDrive/AWS.v39i.seals+holes/ready/valid/labels"

def count_files_in_folder(folder_path):
    """
    Cuenta el número de archivos (no directorios) en la carpeta dada.
    """
    count = 0
    for entry in os.listdir(folder_path):
        full_path = os.path.join(folder_path, entry)
        if os.path.isfile(full_path):
            count += 1
    return count

# Contar archivos en cada carpeta
count1 = count_files_in_folder(folder1)
count2 = count_files_in_folder(folder2)

# Mostrar resultados
print(f"Carpeta 1: {count1} archivos")
print(f"Carpeta 2: {count2} archivos")
print(f"Total de archivos en cada carpeta: Imágenes {count1} + Labels: {count2}")

Carpeta 1: 0 archivos
Carpeta 2: 0 archivos
Total de archivos en cada carpeta: Imágenes 0 + Labels: 0


# 4.1  Importar carpetas desde Google Drive si no están en Rar

In [ ]:


# 2. Definir las carpetas en Google Drive que deseas importar
# Por ejemplo, si tienes dos carpetas: una para imágenes y otra para etiquetas
folders_to_copy = [
    "/content/drive/MyDrive/AWS.v39i.seals+holes/ready"  # Carpeta con imágenes
   # "/content/drive/MyDrive/Tmodel_rar_640_tiles_labels/fotos_elegidas/labels"   # Carpeta con archivos txt
]

# 3. Definir un directorio de destino en Colab donde copiarás estas carpetas
destination_base = "/content/dataset_importado"
import os
os.makedirs(destination_base, exist_ok=True)

# 4. Copiar cada carpeta desde Drive al directorio local
for folder in folders_to_copy:
    folder_name = os.path.basename(folder)
    dest = os.path.join(destination_base, folder_name)
    # El comando cp -r copia recursivamente la carpeta
    !cp -r "{folder}" "{dest}"

# 5. Verificar que se copiaron las carpetas
!ls -l "{destination_base}"

#4.2.  Descomprimir archivos rar de Drive a Google collab directamente si están en rar

In [ ]:


# 2. Instalar unrar para poder descomprimir archivos RAR
!apt-get update
!apt-get install -y unrar



# 4. Definir las rutas de los archivos RAR en Drive
# Ajusta estas rutas según dónde estén ubicados en tu Drive
images_train_rar = "/content/drive/MyDrive/model_rar_640_tiles_labels/train/images.rar"
labels_train_rar = "/content/drive/MyDrive/model_rar_640_tiles_labels/train/labels.rar"
images_validation_rar = "/content/drive/MyDrive/model_rar_640_tiles_labels/valid/images.rar"
labels_validation_rar = "/content/drive/MyDrive/model_rar_640_tiles_labels//valid/labels.rar"

# 5. Definir las carpetas de destino en Colab para cada uno
# Estas carpetas mantendrán la estructura de YOLO (por ejemplo, "images" y "labels")
dest_train_images = "/content/dataset/train"    # he dejado image sy labels en el mismo directorio a ver si así se apaña el no detectar labels
dest_train_labels = "/content/dataset/train"
dest_val_images = "/content/dataset/val"
dest_val_labels = "/content/dataset/val"

import os
os.makedirs(dest_train_images, exist_ok=True)
os.makedirs(dest_train_labels, exist_ok=True)
os.makedirs(dest_val_images, exist_ok=True)
os.makedirs(dest_val_labels, exist_ok=True)

# 6. Descomprimir cada archivo RAR en su carpeta correspondiente
!unrar x "{images_train_rar}" "{dest_train_images}/"
!unrar x "{labels_train_rar}" "{dest_train_labels}/"
!unrar x "{images_validation_rar}" "{dest_val_images}/"
!unrar x "{labels_validation_rar}" "{dest_val_labels}/"

# Opcional: listar el contenido para confirmar que se descomprimió correctamente
!ls /content/dataset/train/       # he dejado image sy labels en el mismo directorio a ver si así se apaña el no detectar labels
!ls /content/dataset/train/
!ls /content/dataset/val/
!ls /content/dataset/val/


#7. WANDB

Conectar Weights & Biases para poder seguir el proceso de aprendizaje

Se guarda todo en la nube de la página web

In [ ]:

# 8. Autenticación en Weights & Biases (W&B)
import wandb
from wandb.integration.ultralytics import add_wandb_callback

# Initialize your Weights & Biases environment
wandb.login(key="105f813f52ef215433a521087f025ed76bf34c23")  # Te pedirá ingresar tu API key o autenticarte desde el navegador.

# Enable Weights & Biases logging
yolo_settings_wandb=True

# Disable Weights & Biases logging
#yolo settings wandb=False


#8.1 Initialize a Weight and Biases run:
wandb.init(project= "seals_2025 ", job_type= "training")



# 8. Mejorar la computación de la GPU, somehow

In [7]:
# 5. Configurar PyTorch para GPU (opcional: optimización extra)
import torch
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True  # mejora la velocidad en convoluciones si el input es fijo

#9. Importar el DATASET Y data.yaml de drive

una vez tenemos todo en Google colab,  para hacer bien los directorios

In [ ]:
################################IMPORTAR EL dataset entero a colab UNA VEZ TIENES LA ESTRUCTURA  ####################################################
####################################################################################


# Copiar el archivo data.yaml desde Drive a la ruta deseada en Colab
!cp -r "/content/drive/MyDrive/AWS.v39i.seals_plus_holes/ready/train/images" "/content/dataset/train/"

# Verificar que se copió correctamente
!ls -l "/content/dataset"

In [ ]:
import os

# Rutas de las dos carpetas
folder1 = "/content/dataset/train/images"
folder2 = "/content/dataset/train/labels"

def count_files_in_folder(folder_path):
    """
    Cuenta el número de archivos (no directorios) en la carpeta dada.
    """
    count = 0
    for entry in os.listdir(folder_path):
        full_path = os.path.join(folder_path, entry)
        if os.path.isfile(full_path):
            count += 1
    return count

# Contar archivos en cada carpeta
count1 = count_files_in_folder(folder1)
count2 = count_files_in_folder(folder2)

# Mostrar resultados
print(f"Carpeta 1: {count1} archivos")
print(f"Carpeta 2: {count2} archivos")
print(f"Total de archivos en cada carpeta: Imágenes {count1} + Labels: {count2}")

para cambiar questionable seal a seal

In [ ]:
import os

def reemplazar_clase_en_txts(ruta_labels, clase_origen=1, clase_destino=2):
    """
    Reemplaza la clase_origen por clase_destino al inicio de cada línea en archivos YOLO .txt.

    Args:
        ruta_labels (str): Carpeta que contiene archivos .txt en formato YOLO.
        clase_origen (int): ID de la clase que quieres reemplazar (por defecto: 1).
        clase_destino (int): ID de la clase que quieres usar en su lugar (por defecto: 2).
    """
    txt_files = [f for f in os.listdir(ruta_labels) if f.endswith(".txt")]

    for file in txt_files:
        file_path = os.path.join(ruta_labels, file)

        with open(file_path, "r") as f:
            lineas = f.readlines()

        nuevas_lineas = []
        for linea in lineas:
            partes = linea.strip().split()
            if partes and partes[0] == str(clase_origen):
                partes[0] = str(clase_destino)
            nuevas_lineas.append(" ".join(partes) + "\n")

        with open(file_path, "w") as f:
            f.writelines(nuevas_lineas)

        print(f"✅ Archivo procesado: {file}")

# 🧪 Ejemplo de uso:
carpeta_labels = "/content/dataset/valid/labels"  # Reemplaza por la ruta correcta
reemplazar_clase_en_txts(carpeta_labels)


#10. Importar YOLO


Impotamos el modelo de Yolo que vamos a usar

Aseguramos que WANDb está conectado

In [ ]:

# 9. Ejecutar el entrenamiento de YOLOv8
from ultralytics import YOLO

# Carga el modelo (en este ejemplo, YOLOv8s)
model = YOLO("yolov8s.pt")

#Add W&B callback for ultralytics:
#add_wandb_callback(model, True, True)



#11. **Train de model**

El paso importante, entrenar al modelo.

Anotar qué parámetros usas para ver max perfomance y cambiar el data.yaml cuando se requiera

In [ ]:
# Ejecuta el entrenamiento utilizando el archivo data.yaml en la carpeta extraída.
model.train(
    project="seals_2025",
    name="focas_prueba_snow_yolov8s_without_ques",
    data="/content/dataset/data.yaml",  # Asegúrate de que data.yaml use rutas correctas para Colab
    epochs=15,                                                  # Número total de épocas (pasadas completas por el dataset de entrenamiento)
    imgsz=640,                                                  # Tamaño de la imagen (se reescala a 640x640)
    batch=16,                                                   # Número de imágenes por batch
    lr0=0.001,        #probar 0.0005                            # Tasa de aprendizaje inicial. Un valor mayor hace que los pesos se actualicen más rápido, pero puede inestabilizar el entrenamiento.
    momentum=0.937,                                             # Momentum para el optimizador (por ejemplo, en SGD)
    warmup_momentum=0.8,                                        # La fase de "warmup" es un período inicial en el que la tasa de aprendizaje y otros parámetros (como el momentum) se ajustan gradualmente.    Esto ayuda a estabilizar el entrenamiento y a evitar que el modelo se desvíe demasiado al principio.
    optimizer="adamw",       #otra opcion es = sgd              # Se usa AdamW (una variante de Adam que incorpora weight decay para evitar sobreajuste)
    weight_decay=0.0005,                                        # Penalización sobre los pesos para prevenir sobreajuste. Si es muy alto, puede dificultar el aprendizaje; si es muy bajo, el modelo puede sobreajustarse.                                          # Se usa AdamW (una variante de Adam que incorpora weight decay para evitar sobreajuste)
    #augment=False,                                             # Desactiva el data augmentation on the fly de YOLO##############  asi puedo hacer yo la data augmentation con las fotos con labels
    half=True,                                                  # Activa precisión mixta (FP16) para GPU
    workers=8,                                                  # Número de workers para el DataLoader
    device='cuda' ,                                            # Fuerza el uso de GPU lo que acelera el entrenamiento
    #mosaic=0,                                                  # Desactivar si las imágenes son muy homogéneas, es un data augmentation process
    #resume=True,                                               #PARA seguir entrenamiento donde lo dejaste
    #validate=False                                              # para no hacer validación a la vez que entrena
    patience=15                                                 # si el modelo no aprende más en la epoch 15, se para. si tras "X" epochs la métrica val/best_map_50 no mejora, se detiene el entrenamiento automáticamente.
)



#12 Gráficas durante aprendizaje en Tensorboard

para poder ver el aprendizaje, llamas a tensorboard

In [ ]:
%load_ext tensorboard

# Iniciar TensorBoard apuntando al directorio donde se guardan los logs
%tensorboard --logdir /content/seals_2025/focas_prueba_snow


# 13. Guardar resultados del modelo entrenado

Tienes que fijarte donde se está guardando, normalmente algo como **runs**

In [12]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [29]:
# 10. Guardar los resultados del experimento

# Define la carpeta de destino en Google Drive
dest_folder = "/content/drive/MyDrive/AWS.v39i.seals_plus_holes/resultados_yolov8s_sinquestionableseal"
!mkdir -p "{dest_folder}"  # El comando mkdir -p crea esa carpeta (y cualquier subcarpeta necesaria) si no existe.

# Copia la carpeta de resultados (ajusta el nombre según el que se haya generado)
!cp -r /content/seals_2025/focas_prueba_snow_yolov8s_without_ques "{dest_folder}/"  # Copiar los resultados del modelo en drive

# Comprueba que se copiaron los archivos (opcional)
!ls "{dest_folder}"

# Opcional: comprimir la carpeta de resultados para descargarla a tu PC
#zip_output_path = "/content/negative_images.zip"  # Ruta temporal para almacenar el archivo ZIP en el entorno de Colab

# Corrige la ruta de compresión para comprimir la carpeta images_negatives correctamente
#!zip -r "{zip_output_path}" "{dest_folder}/images_negatives"  # Aquí se comprime la carpeta que acabas de copiar en Drive (images_negatives) en un archivo ZIP.

# Descargar el archivo ZIP a tu PC
#from google.colab import files
#files.download(zip_output_path)  # Descargar el archivo comprimido                                       #Este comando abre una ventana de descarga para que puedas bajar el archivo ZIP a tu ordenador.

focas_prueba_snow_yolov8s_without_ques


#14  Validación posterior

Aquí puedes ajustar nivel de confianza e IOU para ver la validación del modelo, a pesar d e que lo haga on the fly durante entrenamiento

In [ ]:
# Cargar el modelo entrenado (ajusta la ruta según corresponda)
model = YOLO("/content/seals_2025/focas_prueba_snow/weights/best.pt")

# Ejecutar validación especificando el archivo de datos y los umbrales deseados
metrics = model.val(data="/content/dataset/data.yaml", iou=0.35, conf=0.35)

# Mostrar las métricas obtenidas en la validación
print("Métricas de validación:", metrics)

In [24]:
# 10. Guardar los resultados del experimento

# Define la carpeta de destino en Google Drive
dest_folder = "/content/drive/MyDrive/dataset_selectedplus_snow23_3.yolov8/resultados_yolov8m_without_quest"
!mkdir -p "{dest_folder}"                                                           #El comando mkdir -p crea esa carpeta (y cualquier subcarpeta necesaria) si no existe.

# Copia la carpeta de resultados (ajusta el nombre según el que se haya generado)
!cp -r /content/seals_2025/focas_prueba_snow_yolov8m2 "{dest_folder}/"   #copiar los reultados del modelo en drive


# Comprueba que se copiaron los archivos (opcional)
!ls "{dest_folder}"                                                                 #verifica que los archivos se han copiado correctamente

# Opcional: comprimir la carpeta de resultados para descargarla a tu PC
#!zip -r /content/seals_2025/focas_prueba_snow.zip "{dest_folder}/focas_prueba_snow"   #Aquí se comprime la carpeta que acabas de copiar en Drive (la subcarpeta focas_training_prueba1_640_24_02_25 dentro de foquitas) en un archivo ZIP llamado foquitas.zip que se guarda en /content (la carpeta principal del entorno de Colab).

# Descargar el archivo ZIP a tu PC
#from google.colab import files
#files.download('/content/seals_2025/focas_prueba_snow.zip')                                             #Este comando abre una ventana de descarga para que puedas bajar el archivo ZIP a tu ordenador.

focas_prueba_snow_yolov8m2


#15. Inferencia

Da como resultado, tanto las imagenes con bounding box, como el numero total de elementos por clase

Recuerda conectar DRIVE Y ULTRALYTICS


In [ ]:
import os
import json
import csv
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO

# 1. Cargar el modelo
model_path = "/content/drive/MyDrive/model_rar_640_tiles_labels/Fotos_elegidas/foquitas/focas_prueba_4_images/weights/best.pt"
model = YOLO(model_path)

# 2. Definir carpeta de entrada y salida
input_folder = "/content/drive/MyDrive/model_rar_640_tiles_labels/Fotos_elegidas/test/images"
output_folder = "/content/inference_results_4"
os.makedirs(output_folder, exist_ok=True)

# Dentro del output_folder, creamos dos subcarpetas: with_labels y no_labels
with_labels_folder = os.path.join(output_folder, "with_labels")
no_labels_folder = os.path.join(output_folder, "no_labels")
os.makedirs(with_labels_folder, exist_ok=True)
os.makedirs(no_labels_folder, exist_ok=True)

# 3. Diccionarios para almacenar resultados
results_dict = {}           # Por imagen: { "imagen.jpg": {"seal": 3, "hole": 2} }
global_counts = defaultdict(int)  # Recuento total por clase

# 4. Iterar sobre todas las imágenes en la carpeta de entrada
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(input_folder, filename)


###########################################     Cambiar Parámetros    ######################################################
############################################################################################################################

        # Realizar la inferencia (ajusta los parámetros conf, iou, etc. si es necesario)
        results = model.predict(source=img_path, imgsz=640, conf=0.25, iou=0.2, augment=False)

############################################################################################################################
############################################################################################################################

        # Si hay resultados (normalmente 1 por imagen si se pasa una sola imagen al modelo)
        if len(results) > 0:
            result = results[0]

            # 5. Dibujar las detecciones en la imagen
            rendered_img = result.plot()   # Devuelve una imagen (array numpy en RGB)
            # Convertir de RGB a BGR para guardar con cv2
            rendered_bgr = cv2.cvtColor(rendered_img, cv2.COLOR_RGB2BGR)

            # Determinar si hay al menos una detección
            if result.boxes is not None and len(result.boxes) > 0:
                # Sí hay detecciones
                out_img_folder = with_labels_folder
            else:
                # No hay detecciones
                out_img_folder = no_labels_folder

            # Guardar la imagen con bounding boxes en la carpeta correspondiente
            out_img_path = os.path.join(out_img_folder, filename)
            cv2.imwrite(out_img_path, rendered_bgr)

            # 6. Contar las detecciones por clase
            if result.boxes is not None and len(result.boxes) > 0:
                cls_tensor = result.boxes.cls
                cls_indices = cls_tensor.cpu().numpy().tolist()
                class_names = [model.names[int(idx)] for idx in cls_indices]
            else:
                class_names = []

            count_dict = {}
            for cls_name in class_names:
                count_dict[cls_name] = count_dict.get(cls_name, 0) + 1

            # Guardar en results_dict
            results_dict[filename] = count_dict

            # Acumular en global_counts
            for cls_name, cnt in count_dict.items():
                global_counts[cls_name] += cnt

            print(f"Inferencia realizada: {filename}")
        else:
            print(f"No se obtuvieron resultados para: {filename}")

# 7. Guardar los resultados en CSV y JSON

# CSV: tendremos filas de la forma [filename, class, count] para cada imagen,
# luego una fila "TOTAL" por clase
csv_file = os.path.join(output_folder, "inference_counts.csv")
with open(csv_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "class", "count"])

    # Escribir los datos por imagen
    for fname, counts in results_dict.items():
        if len(counts) == 0:
            # Imagen sin detecciones
            writer.writerow([fname, "no_detections", 0])
        else:
            for cls, cnt in counts.items():
                writer.writerow([fname, cls, cnt])

    # Agregar filas de TOTales (una por clase)
    for cls, cnt in global_counts.items():
        writer.writerow(["TOTAL", cls, cnt])

# JSON: contendrá un objeto con "por_imagen" y "totales"
json_data = {
    "por_imagen": results_dict,
    "totales": dict(global_counts)
}

json_file = os.path.join(output_folder, "inference_counts.json")
with open(json_file, "w") as f:
    json.dump(json_data, f, indent=4)

print("Inferencia completada. Resultados guardados en:")
print(f"- {with_labels_folder} -> imágenes con detecciones")
print(f"- {no_labels_folder} -> imágenes sin detecciones")
print(f"- {csv_file}")
print(f"- {json_file}")


#16. Guardar resultados de la Inferencia

In [ ]:
# Define la carpeta de destino en Google Drive
dest_folder = "/content/drive/MyDrive/model_rar_640_tiles_labels/Fotos_elegidas/resultadosfinales_2_cambiandoiou"
!mkdir -p "{dest_folder}"                                                           #El comando mkdir -p crea esa carpeta (y cualquier subcarpeta necesaria) si no existe.

# Copia la carpeta de resultados (ajusta el nombre según el que se haya generado)
!cp -r /content/inference_results_4 "{dest_folder}/"   #copiar los reultados del modelo en drive


# Comprueba que se copiaron los archivos (opcional)
!ls "{dest_folder}"                                                                 #verifica que los archivos se han copiado correctamente

# Opcional: comprimir la carpeta de resultados para descargarla a tu PC
!zip -r /content/inferencia_2.zip "{dest_folder}/inference_results_4"   #Aquí se comprime la carpeta que acabas de copiar en Drive (la subcarpeta focas_training_prueba1_640_24_02_25 dentro de foquitas) en un archivo ZIP llamado foquitas.zip que se guarda en /content (la carpeta principal del entorno de Colab).

# Descargar el archivo ZIP a tu PC
from google.colab import files
files.download('/content/inferencia_2.zip')                                             #Este comando abre una ventana de descarga para que puedas bajar el archivo ZIP a tu ordenador.